In [3]:
import torch
import os
import sys
sys.path.append(r"C:\Users\Iakov\jupyter\RUDN\Rosneft2025\GNP")
from GNP.nn import ResGCN_AAA

num_layers = 8              # number of layers in GNP
embed = 16                  # embedding dimension in GNP
hidden = 32                 # hidden dimension in MLPs in GNP
drop_rate = 0.0             # dropout rate in GNP
disable_scale_input = False # whether disable the scaling of inputs in GNP
dtype = torch.float32       # training precision for GNP

# Computing device
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

net = ResGCN_AAA(num_layers, embed, hidden, drop_rate,
             scale_input=not disable_scale_input, dtype=dtype).to(device)

for p in range(0, 8):
    P = str(p)
    out_path = f"./dump_{P}/"
    model_name = [f for f in os.listdir(out_path) if f.endswith(".pt")][-1]
    model_file = os.path.join(out_path, model_name)
    net.load_state_dict(torch.load(model_file, map_location=device))
    m = torch.jit.script(net)
    m.save(os.path.join(out_path, "scriptmodule.ts"))

TypeError: ResGCN.__init__() missing 1 required positional argument: 'drop_rate'

In [5]:
torch_script_model = os.path.join(out_path, "scriptmodule.ts")
m = torch.jit.load(torch_script_model, map_location='cpu')

In [10]:
from GNP.utils import scale_A_by_spectral_radius, load_npzsparse

# Load problem
location = r"F:\Rosneft2025\bigdata\matrix"
problem = 50
A, b, x = load_npzsparse(location, problem, device, has_solution=True)

# Normalize A to avoid hassles
A, gamma = scale_A_by_spectral_radius(A)
b = b / gamma

C:\Users\Iakov\jupyter\RUDN\Rosneft2025\GNP\GNP\utils.py:81: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\SparseCsrTensorImpl.cpp:55.)
  A = torch.sparse_csr_tensor(A.indptr, A.indices, A.data, A.shape,


In [19]:
dtype = torch.float32
net = net.to(device)
m = m.to(device)
A = A.to(device).type(dtype)
b = b.to(device).type(dtype)

In [21]:
# net(A, b)
b.dtype

torch.float32

In [24]:
m

RecursiveScriptModule(
  original_name=ResGCN_AAA
  (mlp_initial): RecursiveScriptModule(
    original_name=MLP
    (lin): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=Linear)
      (2): RecursiveScriptModule(original_name=Linear)
      (3): RecursiveScriptModule(original_name=Linear)
    )
    (dropout): RecursiveScriptModule(original_name=Dropout)
  )
  (mlp_final): RecursiveScriptModule(
    original_name=MLP
    (lin): RecursiveScriptModule(
      original_name=ModuleList
      (0): RecursiveScriptModule(original_name=Linear)
      (1): RecursiveScriptModule(original_name=Linear)
      (2): RecursiveScriptModule(original_name=Linear)
      (3): RecursiveScriptModule(original_name=Linear)
    )
    (dropout): RecursiveScriptModule(original_name=Dropout)
  )
  (gconv): RecursiveScriptModule(
    original_name=ModuleList
    (0): RecursiveScriptModule(
      original_name=GCN